# Async Request Lab

In [1]:
import asyncio
import json
import logging
from random import random
import aiohttp

from appvocai.domain.request.appdata import RequestAppData, RequestAppDataGen
from appvocai.domain.request.review import RequestAppReview, RequestAppReviewGen, AsyncRequestAppReview
from appvocai.domain.response.appdata import ResponseAppData
from appvocai.domain.response.review import ResponseAppReview
from appvocai.infra.base.config import Config
from appvocai.toolkit.print import Printer


In [2]:
logger = logging.getLogger(__name__)

## Session

In [3]:
CONCURRENCY=5
TIMEOUT=30
RETRIES = 3

In [4]:
connector = aiohttp.TCPConnector()
concurrency = asyncio.Semaphore(CONCURRENCY)
timeout = aiohttp.ClientTimeout(total=TIMEOUT)

## AppData

In [5]:
CATEGORY = 6018
MAX_REQUESTS = 10
BATCH_SIZE = 2
PAGE = 2
LIMIT = 2
OFFSET = 0
START_IDX = 0


### AppData Generate Requests

## AppReviews

In [6]:
APP_ID = 544007664

In [7]:
# appreview_rgen = RequestAppReviewGen(app_id=APP_ID, max_requests=MAX_REQUESTS, batch_size=BATCH_SIZE, start_page=START_IDX, limit=LIMIT)
# assert isinstance(appreview_rgen, RequestAppReviewGen)

In [8]:
# for arequest in appreview_rgen:
#     assert isinstance(arequest, AsyncRequestAppReview)
#     aresponse = await make_requests(async_request=arequest)
#     for response in aresponse:
#         print(response)
#         break

In [9]:
import aiohttp
import asyncio
import nest_asyncio
import logging
import random
from datetime import datetime
from aiohttp import ClientSession

from appvocai.domain.request.review import RequestAppReview, RequestAppReviewGen, AsyncRequestAppReview

# Apply the nest_asyncio patch to allow nested asyncio.run calls in Jupyter
nest_asyncio.apply()

# Mocked or example settings
RETRIES = 3
concurrency = asyncio.Semaphore(10)  # Limit to 10 concurrent requests
connector = aiohttp.TCPConnector(limit=10)  # Limit TCP connections
timeout = aiohttp.ClientTimeout(total=60)  # 60 seconds timeout


# Replace with your actual request generator
appreview_rgen = RequestAppReviewGen(app_id=APP_ID, max_requests=MAX_REQUESTS, batch_size=BATCH_SIZE, start_page=START_IDX, limit=LIMIT)

async def make_request(session, request, concurrency, throttle: bool = True):
    attempts = 0
    async with concurrency:
        while attempts < RETRIES:
            try:
                aresponse = ResponseAppReview()
                aresponse.parse_request(request=request)

                async with session.get(
                    url=request.baseurl,
                    ssl=False,
                    params=request.params,
                    proxy=Config().proxy
                ) as response:

                    if throttle:
                        await asyncio.sleep(random.random())

                    response.raise_for_status()
                    result = await aresponse.parse_response(response=response)
                    return result

            except aiohttp.ClientError as e:
                logger.warning(f"Attempt {attempts + 1} failed: {e}")
                attempts += 1
                if attempts >= RETRIES:
                    logger.error(f"All {RETRIES} attempts failed for request: {request}")
                    raise
            except Exception as e:
                logger.exception(f"Unexpected error: {e}")
                raise
            else:
                break

async def make_requests(session, async_request):
    tasks = [
        make_request(session=session, request=request, concurrency=concurrency)
        for request in async_request.requests
    ]
    return await asyncio.gather(*tasks, return_exceptions=False)

async def main():
    async with aiohttp.ClientSession(
        connector=connector,
        trust_env=True,
        raise_for_status=True,
        timeout=timeout
    ) as session:

        for arequest in appreview_rgen:
            assert isinstance(arequest, AsyncRequestAppReview)
            aresponse = await make_requests(session=session, async_request=arequest)
            for response in aresponse:
                print(response)

# Running the code in Jupyter
await main()



Failed to parse JSON response: Expecting value: line 1 column 1 (char 0)
ERROR:__main__:Unexpected error: Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "/tmp/ipykernel_619839/3127548050.py", line 43, in make_request
    result = await aresponse.parse_response(response=response)
  File "/home/john/projects/appvocai-acquire/appvocai/domain/response/review.py", line 42, in parse_response
    content = await response.json(content_type=None)
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/site-packages/aiohttp/client_reqrep.py", line 1192, in json
    return loads(stripped.decode(encoding))
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/json/decoder.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

ERROR:__main__:Unexpected error: Session is closed
Traceback (most recent call last):
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/asyncio/sslproto.py", line 534, in data_received
    ssldata, appdata = self._sslpipe.feed_ssldata(data)
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/asyncio/sslproto.py", line 206, in feed_ssldata
    self._sslobj.unwrap()
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/ssl.py", line 979, in unwrap
    return self._sslobj.shutdown()
ssl.SSLError: [SSL: APPLICATION_DATA_AFTER_CLOSE_NOTIFY] application data after close notify (_ssl.c:2702)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/tmp/ipykernel_619839/3127548050.py", line 32, in make_request
    async with session.get(
  File "/home/john/miniconda3/envs/appvocai/lib/python3.10/site-packages/aiohttp/client.py", line 1197, in __aenter__
    self._resp = await self._coro
  File "/home/john/miniconda3/